In [ ]:
%run ../config/init.py

### Creating results folders

In [ ]:
data_dir = os.path.join(RESULTS, DATASET, 'alignments')
result_dir = working_dir(os.path.join(RESULTS, DATASET, 'peak-calling'))
sample_table_file = os.path.join(DATA, DATASET, 'sample_table.csv')
sample_table = pandas.read_csv(sample_table_file, keep_default_na=False)
sample_table.head()

### Peak Calling with MACS2

In [ ]:
log_file = 'peak_calling.log'

peakcalling_yml = {
    'genome_fasta': {'class': 'File', 'path': GENOME_FASTA },
    'genome_gtf': {'class': 'File', 'path': GENOME_GTF },
    'genome_name': GENOME_NAME,
    'macs_callpeaks_g': GENOME_MAPPABLE_SIZE,    
    'macs_callpeaks_q': fdr,
    'nomodel': True,
    'tagAlign_gz': []
}

for c in sample_table['condition'].unique():
    if not os.path.exists(c + '_R0_peaks.narrowPeak'):
        r = os.path.join(data_dir, c + '_R0.tagAlign.gz')
        peakcalling_yml['tagAlign_gz'].append({'class': 'File', 'path': r})
        
for s in sample_table['sample_name'].unique():
    if not os.path.exists(s + '_sorted_peaks.narrowPeak'):
        r = os.path.join(data_dir, s + '_sorted.tagAlign.gz')
        peakcalling_yml['tagAlign_gz'].append({'class': 'File', 'path': r})

if peakcalling_yml['tagAlign_gz']:
    write_to_yaml(peakcalling_yml, 'peakcalling.yml')  
    cmd_header = '{} {}/ChIP-Seq/peak-calling-MACS2.cwl peakcalling.yml > {} 2>&1 &'.format(
        CWLRUNNER, CWLWORKFLOWS, log_file)
    run_command(cmd_header)

### Checking command output
Execute next cell until it prints: **Run completed**

In [ ]:
check_cwl_command_log(log_file)

###  Irreproducible Discovery Rate (IDR)

In [ ]:
log_file = 'idr.log'
data_dir = os.path.join(RESULTS, DATASET, 'peak-calling')
working_dir(os.path.join(RESULTS, DATASET, 'idr'))

idr_yml = {
    'soft_idr_threshold': 0.05,
    'input_file_type': 'narrowPeak',
    'genome_fasta': {'class': 'File', 'path': GENOME_FASTA },
    'genome_gtf': {'class': 'File', 'path': GENOME_GTF },
    'pooled_peak_list': [],
    'narrowpeaks': [],
    'output_file': []
}

for c in sample_table['condition'].unique():    
    ids = sample_table[sample_table['condition'] == c]['sample_name']
    narrowpeaks = []
    for s in ids:
        s = os.path.join(data_dir, s + '_sorted_peaks.narrowPeak')
        narrowpeaks.append({'class': 'File', 'path': s})
    if samples:
        pooled = os.path.join(data_dir, c + '_R0_peaks.narrowPeak') 
        idr_yml['pooled_peak_list'].append({'class': 'File', 'path': pooled})
        idr_yml['output_file'].append(c + '.narrowPeak')
        idr_yml['narrowpeaks'].append(narrowpeaks)
        
if idr_yml['narrowpeaks']:
    write_to_yaml(idr_yml, 'idr.yml')  
    cmd_header = '{} {}/ChIP-Seq/idr.cwl idr.yml > {} 2>&1 &'.format(
        CWLRUNNER, CWLWORKFLOWS, log_file)
    run_command(cmd_header)

### Checking command output
Execute next cell until it prints: **Run completed**

In [ ]:
check_cwl_command_log(log_file)
